In [1]:
from datetime import datetime
import pandas as pd
import yaml
from sqlalchemy import create_engine

## Database Connection

In [2]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [3]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [4]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Load table cliente

In [5]:
tipo_cliente = pd.read_sql_table("tipo_cliente",url_db)
cliente = pd.read_sql_table("cliente",url_db)
ciudad = pd.read_sql_table("ciudad",url_db)
departamento = pd.read_sql_table("departamento",url_db)
auth_user = pd.read_sql_table("auth_user",url_db)

In [6]:
cliente.fillna("no_definido", inplace=True)
cliente.head(5)

C:\Users\nicol\AppData\Local\Temp\ipykernel_27876\170577417.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'no_definido' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  cliente.fillna("no_definido", inplace=True)


,cliente_id,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,ciudad_id,tipo_cliente_id,activo,coordinador_id,sector
0,1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,S
1,2,123,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,2.0,industrial
2,6,24390-3,CLINICA DEPORTIVA DEL SUR,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,1.0,salud
3,19,8301821,HOSPITAL ORTOPEDICO DE COLOMBIA,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,salud
4,8,5017350-8,CLINICA NEFROLOGOS DE CALI,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,salud


In [7]:
# Contar el número de valores nulos por columna
nulos_por_columna = cliente.isnull().sum()
print("Número de valores nulos por columna:\n", nulos_por_columna)

Número de valores nulos por columna:
 cliente_id         0
nit_cliente        0
nombre             0
email              0
direccion          0
telefono           0
nombre_contacto    0
ciudad_id          0
tipo_cliente_id    0
activo             0
coordinador_id     0
sector             0
dtype: int64


In [8]:
tipo_cliente.rename(columns={"nombre": "nombre_tipo_cliente"}, inplace=True)
tipo_cliente


,tipo_cliente_id,nombre_tipo_cliente,descripcion
0,1,Persona Juridica,None
1,2,Persona Natural,None


In [9]:
cliente = cliente.merge(
    tipo_cliente,
    left_on='tipo_cliente_id',  
    right_on='tipo_cliente_id',                   
    how='left'                       
)
cliente.head(5)

,cliente_id,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,ciudad_id,tipo_cliente_id,activo,coordinador_id,sector,nombre_tipo_cliente,descripcion
0,1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,S,Persona Juridica,None
1,2,123,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,2.0,industrial,Persona Juridica,None
2,6,24390-3,CLINICA DEPORTIVA DEL SUR,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,1.0,salud,Persona Juridica,None
3,19,8301821,HOSPITAL ORTOPEDICO DE COLOMBIA,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,salud,Persona Juridica,None
4,8,5017350-8,CLINICA NEFROLOGOS DE CALI,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,salud,Persona Juridica,None


In [10]:
ciudad.rename(columns={"nombre": "nombre_ciudad"}, inplace=True)
ciudad.head(5)

,ciudad_id,nombre_ciudad,departamento_id
0,6,BUGA,1
1,5,BOGOTA,2
2,4,PASTO,4
3,3,POPAYAN,3
4,2,PALMIRA,1


In [11]:
departamento.rename(columns={"nombre": "nombre_departamento"}, inplace=True)
departamento

,departamento_id,nombre_departamento
0,4,NARIÑO
1,3,CAUCA
2,2,CUNDINAMARCA
3,1,VALLE DEL CAUCA


In [12]:
departamento.rename(columns={"id": "id_user"}, inplace=True)
auth_user.head(5)

,id,password,last_login,is_superuser,username,first_name,last_name,email,is_staff,is_active,date_joined
0,318,pbkdf2_sha256$600000$zkyZt2EE1IbKK2BEQIh0qg$cA...,NaT,False,Juan paz,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,False,True,2023-08-31 15:58:54+00:00
1,322,pbkdf2_sha256$600000$0PRLEKfGpLgULMpmcShBQN$nR...,2024-08-29 14:20:34.103482+00:00,False,Luis Rubio,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,False,True,2023-09-04 20:48:28+00:00
2,326,pbkdf2_sha256$600000$a4qFxPBCprNQVGCKzoV4zt$vj...,2024-05-23 11:19:34.968180+00:00,False,Alvaro Dominguez,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,False,True,2023-09-26 22:27:31+00:00
3,327,pbkdf2_sha256$600000$v9Zh6S0AKTvSqRwUnD67gw$8W...,2024-08-08 20:28:39.962222+00:00,False,Luis Castro,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,False,True,2023-10-05 15:34:53+00:00
4,328,pbkdf2_sha256$600000$kzNKcQLO72lUwSElgSWzvK$u1...,NaT,False,Luis Valenciano,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,False,False,2023-10-05 15:40:15+00:00


In [13]:
cliente = cliente.merge(
    ciudad,
    left_on='ciudad_id',  
    right_on='ciudad_id',                   
    how='left'                       
)
cliente.head()

,cliente_id,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,ciudad_id,tipo_cliente_id,activo,coordinador_id,sector,nombre_tipo_cliente,descripcion,nombre_ciudad,departamento_id
0,1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,S,Persona Juridica,None,CALI,1
1,2,123,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,2.0,industrial,Persona Juridica,None,CALI,1
2,6,24390-3,CLINICA DEPORTIVA DEL SUR,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,1.0,salud,Persona Juridica,None,CALI,1
3,19,8301821,HOSPITAL ORTOPEDICO DE COLOMBIA,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,salud,Persona Juridica,None,CALI,1
4,8,5017350-8,CLINICA NEFROLOGOS DE CALI,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,salud,Persona Juridica,None,CALI,1


In [14]:
cliente = cliente.merge(
    departamento,
    left_on='departamento_id',  
    right_on='departamento_id',                   
    how='left'                       
)
cliente.head()

,cliente_id,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,ciudad_id,tipo_cliente_id,activo,coordinador_id,sector,nombre_tipo_cliente,descripcion,nombre_ciudad,departamento_id,nombre_departamento
0,1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,S,Persona Juridica,None,CALI,1,VALLE DEL CAUCA
1,2,123,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,2.0,industrial,Persona Juridica,None,CALI,1,VALLE DEL CAUCA
2,6,24390-3,CLINICA DEPORTIVA DEL SUR,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,1.0,salud,Persona Juridica,None,CALI,1,VALLE DEL CAUCA
3,19,8301821,HOSPITAL ORTOPEDICO DE COLOMBIA,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,salud,Persona Juridica,None,CALI,1,VALLE DEL CAUCA
4,8,5017350-8,CLINICA NEFROLOGOS DE CALI,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,no_definido,salud,Persona Juridica,None,CALI,1,VALLE DEL CAUCA


In [15]:
cliente.drop(columns=["tipo_cliente_id","ciudad_id","departamento_id","coordinador_id","descripcion"], inplace=True)
cliente["saved"] = datetime.now()
cliente.head()

,cliente_id,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,activo,sector,nombre_tipo_cliente,nombre_ciudad,nombre_departamento,saved
0,1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,True,S,Persona Juridica,CALI,VALLE DEL CAUCA,2024-11-01 19:07:48.397813
1,2,123,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,True,industrial,Persona Juridica,CALI,VALLE DEL CAUCA,2024-11-01 19:07:48.397813
2,6,24390-3,CLINICA DEPORTIVA DEL SUR,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,True,salud,Persona Juridica,CALI,VALLE DEL CAUCA,2024-11-01 19:07:48.397813
3,19,8301821,HOSPITAL ORTOPEDICO DE COLOMBIA,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,True,salud,Persona Juridica,CALI,VALLE DEL CAUCA,2024-11-01 19:07:48.397813
4,8,5017350-8,CLINICA NEFROLOGOS DE CALI,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,True,salud,Persona Juridica,CALI,VALLE DEL CAUCA,2024-11-01 19:07:48.397813


## Load table into ETL database

In [ ]:
cliente.to_sql("dim_cliente", etl_conn, index_label="key_cliente", if_exists="replace") # Contar el número de valores nulos por columna